# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "sk-E0y9LTOvDNUAiMvy2h_bFVEmM2qwIL2KERn_Gh8QpMT3BlbkFJdkm7yZShaZQR-0oAPgYo6r_EEL6PYLt1Qar-McvTsA"

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [2]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500)

In [3]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "性能卓越的GPU"
    }))

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




1. PeakTech Graphics Ltd. 
2. EliteGlow Technologies Inc.
3. HyperVoid Processing Co.
4. SuperiorStream GPU Solutions
5. TitanForge Circuitry Corporation
6. QuantumVortex Semiconductors
7. NovaCore Dynamics Ltd.
8. BlazePixel Technologies Inc.
9. RadianceLogic Design Company
10. ApexFlow Graphics Engineering Ltd.


In [5]:
chain.verbose = True

In [6]:
chain.verbose

True

In [7]:
print(chain.run({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.



1. 火迅科技有限公司（独具创新的GPU制造商）
2. 极晶科技有限公司（打造精密的GPU科技）
3. 烈焰芯片有限公司（激情打造高性能GPU）
4. 梦龙科技有限公司（引领GPU科技的梦想之龙）
5. 雷神芯片有限公司（无可匹敌的GPU强力）
6. 闪电科技有限公司（闪耀出世界级的GPU）
7. 暴风科技有限公司（风靡全球的GPU技术）
8. 星空芯片有限公司（星际般无限可能的GPU）
9. 虎牙科技有限公司（不可小觑的GPU之王）
10. 钛龙科技有限公司（坚不可摧的强劲GPU）


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [8]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [10]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [11]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


在一个遥远的宇宙中，存在着一个拥有强大科技的三体文明。他们曾经被称为无敌的战士，无人能够战胜他们。然而，当他们遇到一群勇敢的人类时，一切都发生了改变。这些人类并不拥有高科技，但他们有着坚定的信念和无畏的勇气。他们不相信三体人的无敌，也不愿意被压迫和奴役。于是，一场人类和三体人之间的战争开始了，双方都不愿意退缩。在这场战争中，谁能够获胜，谁又将会是最终的胜利者？这部戏剧将会探讨人类的勇气和信念，以及在面对强大敌人时的无畏精神。它也将带给观众们一个深刻的思考，即在面对困难和挑战时，我们能否像这些人类一样，坚持不懈地奋斗，不断寻求胜利的可能性。因为无论是三体人，还是人类，都不是无法战胜的。


《无敌的战士》是一部令人印象深刻的戏剧作品，它讲述了一群勇敢的人类与拥有强大科技的三体文明之间的战争。这部剧不仅仅是一场宇宙间的战争，更是关于信念和勇气的较量。

通过对比人类与三体人的不同，剧中展现了人类的无畏精神和不屈的意志。尽管面对强大的敌人，人类并没有被吓倒，相反，他们坚信自己的信念，勇敢地面对挑战。与此同时，剧中也展现了三体文明的强大科技，却缺乏人类所拥有的信念和勇气。这样的对比让观众们思考，究竟是什么决定了胜利和失败，是科技还是信念？

在剧中，人类与三体人之间的战争也反映出了现实生活中的挑战。人类面对的困难和挑战也许并不像剧中那么极端，但是我们能从中得到启示——在面对困难和挑战时，信念和勇气是我们战胜困难的关键。《无敌的战士》让观众们深刻地思考自己的信念和勇气，以及在面对挑战时应该如何坚持不懈，寻求胜利的可能性。

总的来说，这部戏剧充满了想象力和深刻的内涵，它让观众们不仅仅是享受精彩的剧情，更是思考自己的生活和价值观。强烈推荐给所有戏剧爱好者和思考者。

> Finished chain.


In [12]:
review = overall_chain.run("星球大战第九季")



> Entering new SimpleSequentialChain chain...


在遥远的未来，银河系陷入了一场前所未有的战争，一场由不同星球的势力和种族发起的星球大战。在这场战争中，每个人都为了自己的利益和理念而奋斗，没有人能够预料到最终的结局。在第八季的最后，反抗军和第一秩序之间的决战带来了巨大的变化，但也留下了许多未解之谜。

第九季开始，反抗军的领袖雷伊和凯洛·伦的命运仍然未知。他们是否能够团结一致，为了共同的目标而战斗？同时，神秘的帝国残余势力开始崛起，他们的目的是什么？更多的种族和势力加入了战争，让整个银河系更加混乱。

在这场星球大战的最终章中，谁将会成为最终的胜利者？是光明还是黑暗将主宰银河系的未来？所有的谜题将会在这场决战中揭晓。无论是谁，都将为了自己的信念而战，为了星球的命运而奋斗。这是一场关于力量、勇气和牺牲的史诗般的战争，也将成为人们永远不会忘记的传奇。


《星球大战：最终章》是一部宏大的史诗般的戏剧作品，它将观众带入了一个遥远的未来，探索了人类内心的黑暗与光明。这部剧集不仅仅是一部关于星际战争的故事，更是关于人性的探讨。

在第八季的决战中，反抗军和第一秩序之间的对抗令人震撼，但也留下了许多未解之谜。第九季的开篇，更是让观众充满了期待，随着雷伊和凯洛·伦的命运未知，神秘的帝国残余势力的出现，以及更多种族和势力的加入，整个银河系的命运变得更加混乱。

在这场决战中，每个人都为了自己的信念而战，为了星球的命运而奋斗。观众将会被这些角色的勇气和牺牲所感动，也将被剧情的复杂性和惊险的战斗所吸引。

《星球大战：最终章》不仅仅是一部戏剧作品，更是一部探讨人性的作品。它将引发观众对于自我、信念和命运的思考。无论最终的结局如何，这部剧集都将成为人们永远不会忘记的传奇。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [13]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [14]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [15]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [16]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


《三体人不是无法战胜的》是一部以二十一世纪的新中国为背景的戏剧作品。在这个时代，人类社会已经发展到了一个新的高度，科技的进步让人类可以探索更广阔的宇宙。然而，与此同时，地球上也发生了一些奇怪的现象。一群来自外太空的三体人突然出现在地球上，他们拥有超强的科技实力，让人类陷入了前所未有的危机中。

这群三体人并非来自和平的目的，而是想要征服地球，将人类当作他们的奴隶。面对这场前所未有的挑战，人类社会陷入了分裂和混乱，一些人选择投降，而另一些人则选择抵抗。在这样的情况下，一个小小的科学家决定挺身而出，带领人类与三体人进行最后的抗争。

在战斗的过程中，人类发现了三体人的弱点，并利用自己的智慧和勇气，最终成功击败了入侵者。最终，人类团结一心，重建了一个更加美好的社会，也让世界上再也没有了无法战胜的敌人。

《三体人不是无法战胜的》是一部关于人类勇气和智慧的故事，也是对人类团结和进步的赞颂。它将带领观众进入一个充满想象力和冒险的未来世界，同时也提醒我们，只要我们团结一心，就没有什么是无法战胜的。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n《三体人不是无法战胜的》是一部以二十一世纪的新中国为背景的戏剧作品。在这个时代，人类社会已经发展到了一个新的高度，科技的进步让人类可以探索更广阔的宇宙。然而，与此同时，地球上也发生了一些奇怪的现象。一群来自外太空的三体人突然出现在地球上，他们拥有超强的科技实力，让人类陷入了前所未有的危机中。\n\n这群三体人并非来自和平的目的，而是想要征服地球，将人类当作他们的奴隶。面对这场前所未有的挑战，人类社会陷入了分裂和混乱，一些人选择投降，而另一些人则选择抵抗。在这样的情况下，一个小小的科学家决定挺身而出，带领人类与三体人进行最后的抗争。\n\n在战斗的过程中，人类发现了三体人的弱点，并利用自己的智慧和勇气，最终成功击败了入侵者。最终，人类团结一心，重建了一个更加美好的社会，也让世界上再也没有了无法战胜的敌人。\n\n《三体人不是无法战胜的》是一部关于人类勇气和智慧的故事，也是对人类团结和进步的赞颂。它将带领观众进入一个充满想象力和冒险的未来世界，同时也提醒我们，只要我们团结一心，就没有什么是无法战胜的。',
 'review': '\n\n《三体人不是无法战胜的》是一部让人眼前一亮的戏剧作品。它将现实与科幻融合得恰到好处，给观众带来了一场视听盛宴。\n\n剧中的故事情节紧凑，充满惊险和刺激。从人类面临的前所未有的危机，到最终的胜利，每一个场景都让人紧张又充满期待。同时，剧中展现的科幻元素也让人惊叹不已，让人们对未来的世界充满了想象。\n\n除了精彩的剧情，该剧也有着出色的表演。每个演员都将自己的角色演绎得淋漓尽致，特别是小科学家的角色，他的勇气和智慧令人佩服。观众也会被演员们的精湛表演所感染，与剧中的角色一起经历他们的挑战和胜利。\n\n该剧的主题也非常明确，即人类团结和进步的力量。它让观众意识到，只要我们团结一心，就没有什么是无法战胜的。这也是一种对现实社会的启示，让我们反思如何团结起来，共同面对挑战。\n\n总的来说，《三体人不是无法战胜的》是一部兼具科幻、冒险和启示意义的戏剧作品。它将带领观众进入一个充满想象力和冒险的未来世界，同时也提醒我们，只要我们团结一心，就没有什么是无法战胜的。强烈推荐观看！'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果